In [ ]:
!pip install torch torchvision numpy pillow tqdm matplotlib

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from torchvision.utils import save_image
from PIL import Image
import os

# Generator (U-Net)
class Generator(nn.Module):
    def __init__(self, input_channels=3, output_channels=3):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(input_channels, 64, kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(64, output_channels, kernel_size=4, stride=2, padding=1),
            nn.Tanh()
        )

    def forward(self, x):
        return self.model(x)

# Discriminator (PatchGAN)
class Discriminator(nn.Module):
    def __init__(self, input_channels=3):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(input_channels, 64, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2),
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2),
            nn.Conv2d(128, 1, kernel_size=4, stride=1, padding=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

In [2]:
class SunsetDataset(Dataset):
    def __init__(self, sunset_path, daytime_path, transform):
        self.sunset_images = sorted(os.listdir(sunset_path))
        self.daytime_images = sorted(os.listdir(daytime_path))
        self.sunset_path = sunset_path
        self.daytime_path = daytime_path
        self.transform = transform

    def __len__(self):
        return min(len(self.sunset_images), len(self.daytime_images))

    def __getitem__(self, idx):
        sunset_img = Image.open(os.path.join(self.sunset_path, self.sunset_images[idx])).convert("RGB")
        daytime_img = Image.open(os.path.join(self.daytime_path, self.daytime_images[idx])).convert("RGB")

        sunset_img = self.transform(sunset_img)
        daytime_img = self.transform(daytime_img)

        return sunset_img, daytime_img

# Define Transformations
IMG_SIZE = 256
transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

# Load dataset
dataset = SunsetDataset("sunrise", "daytime", transform)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

In [ ]:
# Losses
criterion_gan = nn.BCELoss()
criterion_cycle = nn.L1Loss()

# Initialize Models
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
generator = Generator().to(device)
discriminator = Discriminator().to(device)

optimizer_g = optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizer_d = optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))

def train_cycleGAN(generator, discriminator, dataloader, epochs):
    for epoch in range(epochs):
        for i, (sunset, daytime) in enumerate(dataloader):
            sunset, daytime = sunset.to(device), daytime.to(device)

            # Train Generator
            optimizer_g.zero_grad()
            fake_daytime = generator(sunset)
            d_output = discriminator(fake_daytime)
            real_labels = torch.ones_like(d_output, device=device)
            fake_labels = torch.zeros_like(d_output, device=device)
            g_loss = criterion_gan(d_output, real_labels) + 100 * criterion_cycle(fake_daytime, daytime)
            g_loss.backward()
            optimizer_g.step()

            # Train Discriminator
            optimizer_d.zero_grad()
            real_output = discriminator(daytime)
            fake_output = discriminator(fake_daytime.detach())
            d_loss = (criterion_gan(real_output, real_labels) + criterion_gan(fake_output, fake_labels)) / 2
            d_loss.backward()
            optimizer_d.step()

            if i % 100 == 0:
                print(f"Epoch [{epoch}/{epochs}], Step [{i}/{len(dataloader)}], D Loss: {d_loss.item()}, G Loss: {g_loss.item()}")

        torch.save(generator.state_dict(), "generator1.pth")
        torch.save(discriminator.state_dict(), "discriminator1.pth")

# Train CycleGAN
train_cycleGAN(generator, discriminator, dataloader, epochs=5)

Epoch [0/1], Step [0/115], D Loss: 0.691335141658783, G Loss: 44.83195495605469
Epoch [0/1], Step [100/115], D Loss: 0.4347599744796753, G Loss: 51.37269973754883


In [7]:
def infer(generator, image_path, transform):
    image = Image.open(image_path).convert("RGB")
    image = transform(image).unsqueeze(0).to(device)
    generator.eval()
    with torch.no_grad():
        corrected_image = generator(image).to(device)
    save_image(corrected_image, "corr_image.png")

# generator.load_state_dict(torch.load("generator.pth"))
# discriminator.load_state_dict(torch.load("discriminator.pth"))

# Run inference
infer(generator, "Image3.jpg", transform)